In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
import librosa as librosa
import warnings
warnings.filterwarnings('ignore')

In [6]:
path = os.getcwd()
path

'/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/src'

In [7]:
p = '/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/'

In [8]:
genres = ['hip-hop', 'classical', 'country', 'electronic', 'metal']

In [9]:
def extract_audio_features(y, sr):
        '''
        This function extracts audio features from an audio file.
                Parameters:
                        id (string): the audio track id 
                        y 
                        sr 
                Returns:
                        audio_features (DataFrame): the extracted audio features
        '''
        # Features to concatenate in the final dictionary
        features = {'chroma_sftf': None, 'rolloff': None, 'zero_crossing_rate': None, 'rmse': None,
                    'flux': None, 'contrast': None, 'flatness': None}

        # Count silence
        if 0 < len(y):
            y_sound, _ = librosa.effects.trim(y)
        features['sample_silence'] = len(y) - len(y_sound)            

        # Using librosa to calculate the features
        features['chroma_sftf'] = np.mean(
            librosa.feature.chroma_stft(y=y, sr=sr))
        features['rolloff'] = np.mean(
            librosa.feature.spectral_rolloff(y, sr=sr))
        features['zero_crossing_rate'] = np.mean(
            librosa.feature.zero_crossing_rate(y))
        features['rmse'] = np.mean(librosa.feature.rms(y))
        features['flux'] = np.mean(librosa.onset.onset_strength(y=y, sr=sr))
        features['contrast'] = np.mean(
            librosa.feature.spectral_contrast(y, sr=sr))
        features['flatness'] = np.mean(librosa.feature.spectral_flatness(y))

        # MFCC treatment
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        
        for idx, v_mfcc in enumerate(mfcc):
            features['avg_mfcc'] = np.mean(v_mfcc)
            
        

        features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
        return features

In [10]:

#     in each genre:

#     iterate to next genre

# add the list as the contents of the row of a dataframe

# create list of features for each song
chroma_sftf = []
flux = [] 
rolloff = []
zero_crossing_rate = []
contrast = []
flatness = []
rmse = []
tempo = []
genre = []
mfcc = []
sr = 22050

# loop thru the list of genres
for i in range(len(genres)):

    data_dir = "data/"+genres[i]
    
    new_path = os.path.join(p,data_dir)
    os.chdir(new_path)
    
#   extract audio features using librosa library
#   store extracted audio features in a dataframe
    count = 0
    for track_num in range(len(os.listdir())):

        audio, amplitude = librosa.load((os.listdir()[track_num]))
        audio = np.array(audio)
        amplitude = np.array(amplitude)
        curr_song_feats = extract_audio_features(audio,sr)
        
        # for each song, append data to respective features list
        chroma_sftf.append(curr_song_feats['chroma_sftf'])
        flux.append(curr_song_feats['flux'])
        rolloff.append(curr_song_feats['rolloff'])
        zero_crossing_rate.append(curr_song_feats['zero_crossing_rate'])
        contrast.append(curr_song_feats['contrast'])
        flatness.append(curr_song_feats['flatness'])
        rmse.append(curr_song_feats['rmse'])
        tempo.append(curr_song_feats['tempo'])
        genre.append(genres[i])
        mfcc.append(curr_song_feats['avg_mfcc'])
        
        count += 1
        
        if count%100 == 0:
            print('100 songs done')
        
        
        
    print('done with ', genres[i])
    
        

        


100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
done with  hip-hop
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
done with  classical
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
done with  country
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
done with  electronic
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
100 songs done
done with  metal


In [24]:
col_feats = list(zip(genre,mfcc,chroma_sftf, flux, rolloff, zero_crossing_rate, contrast,flatness,rmse,tempo))
col_names = ["genre", "mfcc","chroma_sftf", "flux", "rolloff", "zero_crossing_rate", "contrast","flatness","rmse","tempo"]

df = pd.DataFrame(col_feats, columns = col_names)
df.head()

,genre,mfcc,chroma_sftf,flux,rolloff,zero_crossing_rate,contrast,flatness,rmse,tempo
0,hip-hop,4.450824,0.431049,1.365420,4318.922711,0.068300,19.013011,0.025153,0.345805,151.999081
1,hip-hop,-0.377781,0.511142,1.667078,4419.694105,0.062677,18.660331,0.022863,0.176869,123.046875
2,hip-hop,0.259693,0.522985,1.359143,5745.967884,0.159102,20.103879,0.053650,0.276045,95.703125
3,hip-hop,0.297143,0.399623,1.744030,4064.054853,0.060242,21.550626,0.020770,0.365921,95.703125
4,hip-hop,-3.304997,0.288411,1.144687,1368.274351,0.039064,21.798417,0.000360,0.072659,117.453835


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   genre               4376 non-null   object 
 1   mfcc                4376 non-null   float64
 2   chroma_sftf         4376 non-null   float64
 3   flux                4376 non-null   float64
 4   rolloff             4376 non-null   float64
 5   zero_crossing_rate  4376 non-null   float64
 6   contrast            4376 non-null   float64
 7   flatness            4376 non-null   float64
 8   rmse                4376 non-null   float64
 9   tempo               4376 non-null   float64
dtypes: float64(9), object(1)
memory usage: 342.0+ KB


In [32]:
result = df.groupby(['genre'])

In [46]:
result2 = df.groupby('genre').head(761).reset_index(drop=True)
type(result2)

pandas.core.frame.DataFrame

In [58]:
import pickle
save_model = open("df.MusicFeatures", "wb")
pickle.dump(df, save_model)
save_model.close()

print(os.getcwd())
pat = '/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/data'
os.chdir(pat)
# with open(pat, 'wb') as f:
#     pickle.dump(object, f)
    
new_path = os.path.join(pat,data_dir)
df.to_pickle('original_df.pickle')
result2.to_pickle('MusicFeats.pickle')

/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/data/metal


In [61]:

print(os.getcwd())
df.to_pickle('original_df.pickle')
result2.to_pickle('MusicFeats.pickle')

/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/data


In [64]:
print(os.getcwd())
df2 = pd.read_pickle('original_df.pickle')
df3 = pd.read_pickle('MusicFeats.pickle')
df3

/home/aqeelali7/Documents/Galvanize/Capstone-2-Music-Genre-Classifier/Song-Genre-Classification/data


,genre,mfcc,chroma_sftf,flux,rolloff,zero_crossing_rate,contrast,flatness,rmse,tempo
0,hip-hop,4.450824,0.431049,1.365420,4318.922711,0.068300,19.013011,0.025153,0.345805,151.999081
1,hip-hop,-0.377781,0.511142,1.667078,4419.694105,0.062677,18.660331,0.022863,0.176869,123.046875
2,hip-hop,0.259693,0.522985,1.359143,5745.967884,0.159102,20.103879,0.053650,0.276045,95.703125
3,hip-hop,0.297143,0.399623,1.744030,4064.054853,0.060242,21.550626,0.020770,0.365921,95.703125
4,hip-hop,-3.304997,0.288411,1.144687,1368.274351,0.039064,21.798417,0.000360,0.072659,117.453835
...,...,...,...,...,...,...,...,...,...,...
3800,metal,0.811577,0.566339,1.299508,4737.379629,0.112808,18.067332,0.039229,0.211604,86.132812
3801,metal,2.362003,0.493252,1.079446,4911.343929,0.085464,18.721121,0.025655,0.273660,99.384014
3802,metal,-0.901563,0.419331,1.319956,5249.888182,0.118459,19.661044,0.038292,0.108418,112.347147
3803,metal,0.775025,0.561058,0.985969,5249.055499,0.099681,18.164477,0.026418,0.331060,123.046875


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   genre               1062 non-null   object 
 1   mfcc                1062 non-null   float64
 2   chroma_sftf         1062 non-null   float64
 3   flux                1062 non-null   float64
 4   rolloff             1062 non-null   float64
 5   zero_crossing_rate  1062 non-null   float64
 6   contrast            1062 non-null   float64
 7   flatness            1062 non-null   float64
 8   rmse                1062 non-null   float64
 9   tempo               1062 non-null   float64
dtypes: float64(9), object(1)
memory usage: 83.1+ KB
